<a href="https://colab.research.google.com/github/MahayaImad/PhD-Thesis-Privacy-AI/blob/main/Training_experiments_with_grid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Parameters

In [1]:
# "convnext" "densenet" "efficientnet" "efficientnet_v2""inception_resnet_v2""inception_v3""mobilenet_v3" "resnet50" "resnet_v2" "vgg16" "vgg19" "xception"
MODEL_ARCHITECTURE = "efficientnet_v2"
EPOCHS = 10
TRANFERT_LEARNING =True
#==============================#

DATASET = "pathmnist_64"
INPUT_SHAPE = (64, 64, 3)
CLASSES = 9
#==============================#
# param_grid = {
#     "learning_rate": [1e-4, 1e-3],
#     "batch_size": [64],
#     "optimizer": ["adam", "sgd"],
#     "dropout": [0.2, 0.3, 0.4, 0.5],
#     "dense_units": [64, 128, 256]
# }
param_grid = {
    "learning_rate": [1e-3],
    "batch_size": [64],
    "optimizer": ["adam"],
    "dropout": [0.3],
    "dense_units": [64]
}
#==============================#
base_path = '/content/drive/MyDrive/ColabNotebooks/PhD-Thesis-Privacy-AI'

## Importing Libraries

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import tensorflow
import numpy as np
import pandas as pd

from sklearn.model_selection import ParameterGrid
from sklearn.metrics import f1_score, confusion_matrix
from tensorflow.keras.models import load_model
sys.path.append(f'{base_path}/lib')

#importing helpers
from import_pathmnist_dataset import PathMNISTLoader
from create_model import create_model
from import_callbacks import get_callbacks
from helper_saving import save_training

# To save in Excel file
# hyperparams = params

from datetime import datetime
experiment_info = {
    "model" : MODEL_ARCHITECTURE,
    "dataset" : DATASET,
    "mode" : "Local",
    "run_time" : datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Dataset

In [3]:
loader = PathMNISTLoader(filename= DATASET, num_classes= CLASSES, batch_size= 64)
x_train , y_train, x_val, y_val, x_test, y_test = loader.get_datasets()

Train: (89996, 64, 64, 3) (89996, 1)
Validation: (10004, 64, 64, 3) (10004, 1)
Test: (7180, 64, 64, 3) (7180, 1)


## Train and Save Results

In [ ]:
# Initialize an empty list to collect results
results = []

for i,config in enumerate(ParameterGrid(param_grid)):
  model = create_model(
    architecture=MODEL_ARCHITECTURE,
    input_shape=INPUT_SHAPE,
    num_classes=CLASSES,
    dropout_rate=config['dropout'],
    learning_rate=config['learning_rate'],
    optimizer=config['optimizer'],
    dense_units=config['dense_units'],
    freeze_base=TRANFERT_LEARNING
    )
  # Get callbacks
  callbacks = get_callbacks(
      model_name = f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}_{i}',
      checkpoint_dir = os.path.join(base_path, MODEL_ARCHITECTURE,'models'),
      )

  # Train the model
  history = model.fit(
      x= x_train,
      y= y_train,
      batch_size=config['batch_size'],
      validation_data=(x_val, y_val),
      epochs=EPOCHS,
      callbacks=callbacks,
      verbose=2,
  )

  model = load_model(os.path.join(base_path, MODEL_ARCHITECTURE,'models',f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}_{i}.keras'))

  # Predict on the test set
  y_pred_probs = model.predict(x_test)
  y_pred = np.argmax(y_pred_probs, axis=1)
  y_true = y_test.flatten()

  # Metrics
  f1 = f1_score(y_true, y_pred, average='macro')
  conf_matrix = confusion_matrix(y_true, y_pred)
  test_loss, test_accuracy = model.evaluate(x_test, y_test)

  # Get best validation accuracy & loss
  best_val_acc = max(history.history["val_accuracy"])
  best_val_loss = min(history.history["val_loss"])

  # Log all info into a result dictionary
  result_record = {
      "model": MODEL_ARCHITECTURE,
      "run_id": i,
      "test_accuracy": test_accuracy,
      "test_loss": test_loss,
      "best_val_accuracy": best_val_acc,
      "best_val_loss": best_val_loss,
      "f1_score": f1,
      "confusion_matrix": conf_matrix.tolist(),
      **config  # unpack all hyperparameters
  }
  results.append(result_record)
  SAVE_PATH=os.path.join(base_path, MODEL_ARCHITECTURE,'results', f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}_{i}.xlsx')
  save_training(save_path=SAVE_PATH,model=model,history = history,hyperparams=config,experiment_info=experiment_info, f1_score=f1, confusion_matrix=conf_matrix.tolist(),test_accuracy=test_accuracy,test_loss=test_loss)

results_df = pd.DataFrame(results)
results_df.to_excel(os.path.join(base_path, MODEL_ARCHITECTURE,'results', f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}.xlsx'), index=False)


Epoch 1/10
1407/1407 - 120s - 85ms/step - accuracy: 0.3342 - loss: 1.8498 - val_accuracy: 0.3954 - val_loss: 1.6464
Epoch 2/10
1407/1407 - 42s - 30ms/step - accuracy: 0.3634 - loss: 1.7634 - val_accuracy: 0.4054 - val_loss: 1.5944
Epoch 3/10
1407/1407 - 40s - 28ms/step - accuracy: 0.3716 - loss: 1.7343 - val_accuracy: 0.4171 - val_loss: 1.6017
Epoch 4/10
1407/1407 - 41s - 29ms/step - accuracy: 0.3770 - loss: 1.7211 - val_accuracy: 0.4285 - val_loss: 1.5755
Epoch 5/10
1407/1407 - 39s - 28ms/step - accuracy: 0.3822 - loss: 1.6996 - val_accuracy: 0.4329 - val_loss: 1.5770
Epoch 6/10


In [ ]:
save_training(save_path=SAVE_PATH,model=model,history = history,hyperparams=config,experiment_info=experiment_info, f1_score=f1, confusion_matrix=conf_matrix.tolist(),test_accuracy=test_accuracy,test_loss=test_loss)